In [40]:
# Data from Copyright © Global Energy Monitor. Distributed under a Creative Commons Attribution 4.0 International License.
# Recommended Citation: “Global Energy Monitor, Global Oil and Gas Extraction Tracker, March 2024 Release”. When the data set is being shared or adapted and our Creative Commons CC BY 4.0 International license applies, please read the license for attribution requirements. Necessary attribution elements are included in our data download files.
# https://globalenergymonitor.org/projects/global-oil-gas-extraction-tracker/

In [41]:
import pandas as pd
import geojson, json, os, random

class GoogleSheetRow():
    columns = [
        'Enabled', 'Share link identifier', 'Category', 'Name', 'Credits', 'Base layer', 'Custom slider ticks', 'Start date',
        'End date', 'Step', 'URL', 'Scaling', 'Color Scaling', 'Map Type', 'Color', 'External GeoJSON', 'Name Key', 'Show Graph',
        'Graph Title', 'Graph Plot Colors', 'Graph Plots First Visible', 'Graph X-Axis Label', 'Graph Y-Axis Label', 'Graph Y-Axis Min',
        'Graph Y-Axis Max', 'Graph X-Axis Label Interval', 'Legend Content', 'Legend Key', 'Load Data Function', 'Set Data Function', 
        'Number of Levels', 'Number of Attributes', 'Vertex Shader', 'Fragment Shader', 'Draw Function', 'Playback Rate',
        'Master Playback Rate', 'Colormap Src', 'Layer Description', 'Featured Themes', 'Draw Options', 'Set Data Options', 'Extras Options',
        'Draw Order', 'Timeline Type', 'Layer Constraints', 'Mapbox', 'Draw Layer Function'
    ]
    key_values = {}
    def __init__(self, _dict = {}):
        for column in self.columns:
            self.key_values[column] = ''
        for key in _dict.keys():
            if key in self.columns:
                self.key_values[key] = _dict[key]
    def set_column(self, key, value):
        if key in self.columns:
            self.key_values[key] = value
    def print_row(self):
        row = []
        for column in self.columns:
            row.append(self.key_values[column])
        print("\t".join(row))
        


In [42]:
fname = "../data/Global-Oil-and-Gas-Extraction-Tracker-March-2024.xlsx"
excel = pd.ExcelFile('../data/Global-Oil-and-Gas-Extraction-Tracker-March-2024.xlsx')

excel.sheet_names

['Main data', 'Production & reserves', 'About']

In [43]:
df = pd.read_excel(excel, "Main data")

In [44]:
feature_collection = geojson.FeatureCollection(features=[])
max_value = 0
for idx, row in df.iterrows():
    if row['Status'] != 'operating' or pd.isna(row['Production start year']) or pd.isna(row['Latitude']):
        continue
    point = geojson.Point((row['Longitude'], row['Latitude']))
    # Why add a random month to start?
    #   We could be exact and just add the start year, but the visualization ends feeling chunky
    #   Adding a somewhat staggered random start to each inidvidual location makes it seem nicer
    #   Just opinions though. Do w/e you want and pick what you like best
    random_month = random.randrange(1,12)
    random_pad_1 = str(random_month).zfill(2) + '01'    
    random_pad_2 = str(random_month+1).zfill(2) + '01'    

    dates = [str(int(row['Production start year']))+random_pad_1, str(int(row['Production start year']))+random_pad_2, '20240101']
    values = [0,100,100]
    properties = {
        'status': 'operating',
        'timeseries': {
            'dates': dates,
            'values': values
        }
        
    }
    feature = geojson.Feature(geometry=point, properties=properties)
    feature_collection['features'].append(feature)


<class 'int'>
{"geometry": {"coordinates": [16.71667, 48.41667], "type": "Point"}, "properties": {"status": "operating", "timeseries": {"dates": ["19510101", "19510201", "20240101"], "values": [0, 100, 100]}}, "type": "Feature"}
<class 'int'>
{"geometry": {"coordinates": [-39.612, -21.36], "type": "Point"}, "properties": {"status": "operating", "timeseries": {"dates": ["20091101", "20091201", "20240101"], "values": [0, 100, 100]}}, "type": "Feature"}
<class 'int'>
{"geometry": {"coordinates": [-36.889, -10.686], "type": "Point"}, "properties": {"status": "operating", "timeseries": {"dates": ["19691101", "19691201", "20240101"], "values": [0, 100, 100]}}, "type": "Feature"}
<class 'int'>
{"geometry": {"coordinates": [-36.262, -4.899], "type": "Point"}, "properties": {"status": "operating", "timeseries": {"dates": ["19790601", "19790701", "20240101"], "values": [0, 100, 100]}}, "type": "Feature"}
<class 'int'>
{"geometry": {"coordinates": [-39.961, -22.131], "type": "Point"}, "properties

In [45]:
oname = 'data/making_a_marker_map_example_1.geojson'
os.makedirs(os.path.dirname(oname), exist_ok=True)
with open(oname, 'w') as f:
    json.dump(feature_collection, f)


In [46]:
# Copy the data somewhere www accessible
#!scp data/making_a_marker_map_example_1.geojson timemachine2:/t/earthtime.org/app/data/making-earthtime-maps
!scp data/making_a_marker_map_example_1.geojson /t/earthtime.org/app/data/making-earthtime-maps

cp: cannot create regular file '/t/earthtime.org/app/data/making-earthtime-maps/making_a_marker_map_example_1.geojson': Permission denied


In [47]:
# Please try to always add a layer description with minimally a brief explanation of what the visualization shows 
#  and where the data came from.
layer_description = "<h2>Oil and Gas Extraction Sites</h2><p>Sourced from <a href=\"https://globalenergymonitor.org/projects/global-oil-gas-extraction-tracker/\">https://globalenergymonitor.org/projects/global-oil-gas-extraction-tracker/</a></p><p>Displaying operating oil and gas extraction sites.</p>"
# Share link identifiers should be unique. Would be great if a DB just handled this... 
share_link_identifer = 'making_a_marker_map_example_1_' + str(random.getrandbits(16))
category = 'Making EarthTime Maps'
name = 'Oil and Gas Extraction Sites'
# This should be wherever yoou uploaded your data to
url = 'https://tiles.earthtime.org/making-earthtime-maps/making_a_marker_map_example_1.geojson'
layer_args = {
    'Enabled': 'TRUE',
    'Share link identifier': share_link_identifer,
    'Category': category, 
    'Name': name, 
    'Credits': 'Global Energy Monitor', 
    'Base layer': 'mb_openplanetv1', 
    'Start date': '1900',
    'End date': '2024', 
    'URL': url, 
    'Scaling': f'd3.scaleSqrt().domain([0,100]).range([0,10]).clamp(true)', 
    'Layer Description': layer_description,
    'Map Type': 'marker',
    'Load Data Function': 'WebGLVectorTile2.prototype._loadGeojsonData',
    'Set Data Function': 'WebGLVectorTile2.prototype._setMarkerData',
    'Number of Attributes': '14',
    'Vertex Shader': 'WebGLVectorTile2.animatedMarkerVertexShader',
    'Fragment Shader': 'WebGLVectorTile2.markerFragmentShader',
    'Draw Function': 'WebGLVectorTile2.prototype._drawMarker', 
    'Draw Options': '{"pointSize": 10, "linewidth": 2, "mode": "disc"}',
    'Set Data Options': '{ "sizeKey": "timeseries", "fills": [[250, 242, 232, 0.55]], "strokes": [[250, 242, 232, 0.75]]}',
    'Draw Order': '700',
}
row = GoogleSheetRow(layer_args)

In [48]:
# C&P the output of this into a sheet. Would be cool if this just like happened...
row.print_row()

TRUE	making_a_marker_map_example_1_43170	Making EarthTime Maps	Oil and Gas Extraction Sites	Global Energy Monitor	mb_openplanetv1		1900	2024		https://tiles.earthtime.org/making-earthtime-maps/making_a_marker_map_example_1.geojson	d3.scaleSqrt().domain([0,100]).range([0,10]).clamp(true)		marker															WebGLVectorTile2.prototype._loadGeojsonData	WebGLVectorTile2.prototype._setMarkerData		14	WebGLVectorTile2.animatedMarkerVertexShader	WebGLVectorTile2.markerFragmentShader	WebGLVectorTile2.prototype._drawMarker				<h2>Oil and Gas Extraction Sites</h2><p>Sourced from <a href="https://globalenergymonitor.org/projects/global-oil-gas-extraction-tracker/">https://globalenergymonitor.org/projects/global-oil-gas-extraction-tracker/</a></p><p>Displaying operating oil and gas extraction sites.</p>		{"pointSize": 10, "linewidth": 2, "mode": "disc"}	{ "sizeKey": "timeseries", "fills": [[250, 242, 232, 0.55]], "strokes": [[250, 242, 232, 0.75]]}		700				


In [49]:
fname2 = "../data/FloodArchive.xlsx"
excel2 = pd.ExcelFile('../data/FloodArchive.xlsx')

excel2.sheet_names

['Sheet 1']

In [50]:
df2 = pd.read_excel(excel2)

In [56]:
feature_collection2 = geojson.FeatureCollection(features=[])
max_value = 0
for idx, row in df2.iterrows():
    if pd.isna(row['Began']) or pd.isna(row['Latitude']):
        continue
    point = geojson.Point((row['Longitude'], row['Latitude']))

    dates = [str(int(row['Began'])), str(int(row['Began'])), '20211009']
    values = [0,100,100]
    properties = {
        'timeseries': {
            'dates': dates,
            'values': values
        }
        
    }
    feature = geojson.Feature(geometry=point, properties=properties)
    feature_collection2['features'].append(feature)


{"geometry": {"coordinates": [5.23026, 35.8142], "type": "Point"}, "properties": {"timeseries": {"dates": ["19850101", "19850101", "20211009"], "values": [0, 100, 100]}}, "type": "Feature"}
{"geometry": {"coordinates": [-45.3489, -18.7111], "type": "Point"}, "properties": {"timeseries": {"dates": ["19850115", "19850115", "20211009"], "values": [0, 100, 100]}}, "type": "Feature"}
{"geometry": {"coordinates": [122.974, 10.0207], "type": "Point"}, "properties": {"timeseries": {"dates": ["19850120", "19850120", "20211009"], "values": [0, 100, 100]}}, "type": "Feature"}
{"geometry": {"coordinates": [124.606, 1.01489], "type": "Point"}, "properties": {"timeseries": {"dates": ["19850204", "19850204", "20211009"], "values": [0, 100, 100]}}, "type": "Feature"}
{"geometry": {"coordinates": [32.3491, -25.8693], "type": "Point"}, "properties": {"timeseries": {"dates": ["19850209", "19850209", "20211009"], "values": [0, 100, 100]}}, "type": "Feature"}
{"geometry": {"coordinates": [43.36, -11.6516],

In [57]:
oname2 = 'data/making_a_marker_map_example_2.geojson'
os.makedirs(os.path.dirname(oname2), exist_ok=True)
with open(oname2, 'w') as f:
    json.dump(feature_collection2, f)


In [61]:
# Copy the data somewhere www accessible
#!scp data/making_a_marker_map_example_1.geojson timemachine2:/t/earthtime.org/app/data/making-earthtime-maps
!scp data/making_a_marker_map_example_2.geojson /t/earthtime.org/app/data/making-earthtime-maps

In [65]:
# Please try to always add a layer description with minimally a brief explanation of what the visualization shows 
#  and where the data came from.
layer_description2 = "<h2>Flood Sites</h2><p>Sourced from <a href=\"https://floodobservatory.colorado.edu/Archives/\">https://floodobservatory.colorado.edu/Archives/</a></p><p>Displaying flood sites.</p>"
# Share link identifiers should be unique. Would be great if a DB just handled this... 
share_link_identifer2 = 'making_a_marker_map_example_2_' + str(random.getrandbits(16))
category2 = 'Making EarthTime Maps'
name2 = 'Flood Sites'
# This should be wherever yoou uploaded your data to
url2 = 'https://tiles.earthtime.org/making-earthtime-maps/making_a_marker_map_example_2.geojson'
layer_args2 = {
    'Enabled': 'TRUE',
    'Share link identifier': share_link_identifer2,
    'Category': category2, 
    'Name': name2, 
    'Credits': 'Dartmouth Flood Observatory', 
    'Base layer': 'mb_openplanetv1', 
    'Start date': '1985',
    'End date': '2021', 
    'URL': url2, 
    'Scaling': f'd3.scaleSqrt().domain([0,100]).range([0,10]).clamp(true)', 
    'Layer Description': layer_description2,
    'Map Type': 'marker',
    'Load Data Function': 'WebGLVectorTile2.prototype._loadGeojsonData',
    'Set Data Function': 'WebGLVectorTile2.prototype._setMarkerData',
    'Number of Attributes': '14',
    'Vertex Shader': 'WebGLVectorTile2.animatedMarkerVertexShader',
    'Fragment Shader': 'WebGLVectorTile2.markerFragmentShader',
    'Draw Function': 'WebGLVectorTile2.prototype._drawMarker', 
    'Draw Options': '{"pointSize": 10, "linewidth": 2, "mode": "disc"}',
    'Set Data Options': '{ "sizeKey": "timeseries", "fills": [[250, 242, 232, 0.55]], "strokes": [[250, 242, 232, 0.75]]}',
    'Draw Order': '700',
}
row = GoogleSheetRow(layer_args2)

In [66]:
# C&P the output of this into a sheet. Would be cool if this just like happened...
row.print_row()

TRUE	making_a_marker_map_example_2_1702	Making EarthTime Maps	Flood Sites	Dartmouth Flood Observatory	mb_openplanetv1		19850101	20211009		https://tiles.earthtime.org/making-earthtime-maps/making_a_marker_map_example_2.geojson	d3.scaleSqrt().domain([0,100]).range([0,10]).clamp(true)		marker															WebGLVectorTile2.prototype._loadGeojsonData	WebGLVectorTile2.prototype._setMarkerData		14	WebGLVectorTile2.animatedMarkerVertexShader	WebGLVectorTile2.markerFragmentShader	WebGLVectorTile2.prototype._drawMarker				<h2>Flood Sites</h2><p>Sourced from <a href="https://floodobservatory.colorado.edu/Archives/">https://floodobservatory.colorado.edu/Archives/</a></p><p>Displaying flood sites.</p>		{"pointSize": 10, "linewidth": 2, "mode": "disc"}	{ "sizeKey": "timeseries", "fills": [[250, 242, 232, 0.55]], "strokes": [[250, 242, 232, 0.75]]}		700				
